In [3]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as ts
import matplotlib.pyplot as plt

In [4]:
df_pos_full = pd.read_csv('comments_positive.csv')
print('Originial', df_pos_full.shape)
df_pos_full.dropna(axis=0, inplace=True)
print('NaN removed',df_pos_full.shape)

Originial (2000000, 15)
NaN removed (1999977, 15)


In [6]:
df_neg_full = pd.read_csv('comments_negative.csv')
print('Originial', df_neg_full.shape)
df_neg_full.dropna(axis=0, inplace=True)
print('NaN removed',df_neg_full.shape)

Originial (2000000, 15)
NaN removed (1999951, 15)


In [7]:
df_pos_full.head()

,id,parent_id,subreddit_id,link_id,text,score,ups,author,controversiality,parent_link_id,parent_text,parent_score,parent_ups,parent_author,parent_controversiality
0,c092j8m,t1_c092gss,t5_2qh2p,t3_8eyy3,This isn't Twitter: try to comment on the arti...,9582,9582,nraustinii,0,t3_8eyy3,Fucking faggot.,-7526,-7526,Glorificus,0
1,c4imcva,t1_c4im948,t5_2qh1i,t3_t0ynr,"Well, it is exactly what it sounds like. It's ...",9531,9531,Lynfect,0,t3_t0ynr,"Elaborate on this cum box, please.",3841,3841,eeeeevil,0
2,c0s4nfi,t1_c0s4lje,t5_2qh1i,t3_cf1n2,"In soviet Russia, bomb disarms you!",8545,8545,CapnScumbone,0,t3_cf1n2,"I don't live in Russia anymore, and I will not...",621,621,shady8x,0
3,c4ini33,t1_c4incln,t5_2qh1i,t3_t0ynr,"""runin for senitur! #YOLO!""",7430,7430,[deleted],0,t3_t0ynr,This just made me realize that future presiden...,4651,4651,drspg99,0
4,c4imgel,t1_c4ima2e,t5_2qh1i,t3_t0ynr,You step motherfucker.,7173,7173,jbg89,0,t3_t0ynr,I have sex with my step mom when my dad isn't ...,4251,4251,audir8,0


In [8]:
df_pos = df_pos_full.sample(frac=0.05,random_state=200)
df_neg = df_neg_full.sample(frac=0.05,random_state=200)
df_pos = df_pos.reset_index(drop=True)
df_neg = df_neg.reset_index(drop=True)

print(df_pos.shape)
print(df_neg.shape)

(99999, 15)
(99998, 15)


In [9]:
num_words_pos = df_pos['text'].apply(lambda x: len(x.split()))
num_words_neg = df_neg['text'].apply(lambda x: len(x.split()))

pos_words_mean, pos_words_std = np.mean(num_words_pos), np.std(num_words_pos)
neg_words_mean, neg_words_std = np.mean(num_words_neg), np.std(num_words_neg)

print("Positive stats:", pos_words_mean, pos_words_std)
print("Negative stats:", neg_words_mean, neg_words_std)

Positive stats: 27.603346033460333 57.202432451660734
Negative stats: 32.63535270705414 51.34923962338161


In [10]:
def clean(text, stemming=False, stop_words=True):
    import re
    from string import punctuation
    from nltk.stem import SnowballStemmer
    from nltk.corpus import stopwords
    from nltk import word_tokenize
    
    stops = stopwords.words('english')
    
    # Empty comment
    if type(text) != str or text=='':
        return ''
    
    # Commence the cleaning!
    urls = r'http(s)*:\/\/(\w|\.)+(\/\w+)*'
    text = re.sub(urls, '', text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are", text)
    text = re.sub("\'ve", " have", text)
    text = re.sub("\'d", " would", text)
    text = re.sub("cant", "can not", text)
    text = re.sub("can\'t", "can not", text)
    text = re.sub("isn\'t", "is not", text)
    text = re.sub("isnt", "is not", text)
    text = re.sub("whats", "what is", text)
    text = re.sub("what\'s", "what is", text)
    text = re.sub("shouldn't", "should not", text, flags=re.IGNORECASE)
    text = re.sub("I'm", "I am", text)
    text = re.sub(":", " ", text)
    # The comments contain \n for line breaks, we need to remove those too
    text = re.sub("\\n", " ", text)
    
    # Special characters
    text = re.sub('\&', " and ", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    
    # Remove punctuation
    text = ''.join([word for word in text if word not in punctuation]).lower()
    
    # If we want to do stemming...
    if stemming:
        sno = SnowballStemmer('english')
        text = ''.join([sno.stem[word] for word in text])
    
    # If we want to remove stop words...
    if stop_words:
        text = text.split()
        text = [word for word in text if word not in stops]
        text = ' '.join(text)
    
    return text

In [11]:
def posneg(number):
    if number > 0:
        return 1
    else:
        return 0

In [12]:

df_pos['text'] = df_pos['text'].apply(clean).astype(str)
df_pos['parent_text'] = df_pos['parent_text'].apply(clean).astype(str)
df_neg['text'] = df_neg['text'].apply(clean).astype(str)
df_neg['parent_text'] = df_neg['parent_text'].apply(clean).astype(str)

df_pos['score'] = df_pos['score'].apply(posneg)
df_pos['parent_score'] = df_pos['parent_score'].apply(posneg)
df_neg['score'] = df_neg['score'].apply(posneg)
df_neg['parent_score'] = df_neg['parent_score'].apply(posneg)

col_write = ['text', 'score', 'ups', 'controversiality', 'parent_text', 'parent_score', 'parent_ups', 'parent_controversiality']
df_pos_clean = df_pos[col_write].copy()
df_neg_clean = df_neg[col_write].copy()
df_pos_clean.dropna(axis=0, inplace=True)
df_neg_clean.dropna(axis=0, inplace=True)
df_pos_clean.to_csv('clean_positive_train.csv', columns=col_write, index=False)
df_neg_clean.to_csv('clean_negative_train.csv', columns=col_write, index=False)

In [13]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [24]:
a =open(('clean_positive_train.csv'), encoding='utf-8')

In [26]:
b = open(('clean_negative_train.csv'), encoding='utf-8')

In [27]:
df_pos = pd.read_csv(a)

In [28]:
df_neg = pd.read_csv(b)

In [30]:
df_pos['text'] = df_pos['text'].astype(str)
df_pos['parent_text'] = df_pos['parent_text'].astype(str)

df_neg['text'] = df_neg['text'].astype(str)
df_neg['parent_text'] = df_neg['parent_text'].astype(str)

In [31]:
df = pd.concat([df_pos, df_neg])
df = df.sample(frac=1).reset_index(drop=True)
df.dropna(axis=0, inplace=True)

In [32]:
df['combined'] = df[['text', 'parent_text']].apply(lambda x: ' '.join(x), axis=1)

text_data = df['combined']
text_score = df['score']
# parent_text_data = df['parent_text']
# parent_text_score = df['parent_score']
X_train, X_test, y_train, y_test = train_test_split(text_data,text_score, test_size = 0.20, random_state = 42)

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

X_train: (159997,)
X_test: (40000,)
y_train: (159997,)
y_test: (40000,)


In [35]:
tokenizer = Tokenizer(num_words=10000, lower=True, split=' ', document_count=0)

# Create the word_index list based on all our data
tokenizer.fit_on_texts(text_data)

# Now we make a list of sequences of integers based on our texts
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [36]:
train_data = keras.preprocessing.sequence.pad_sequences(X_train_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=128)

test_data = keras.preprocessing.sequence.pad_sequences(X_test_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=128)

In [37]:
print(len(train_data[0]), len(train_data[1]))

128 128


In [38]:
vocab_size = len(tokenizer.word_index) + 1

model = tf.keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, 4))
model.add(keras.layers.GlobalAveragePooling1D())
#model.add(keras.layers.Dense(4, activation=tf.nn.relu))
#model.add(keras.layers.Dense(8, activation=tf.nn.relu))
model.add(keras.layers.Dense(2, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 4)           868984    
_________________________________________________________________
global_average_pooling1d (Gl (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 868,997
Trainable params: 868,997
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [40]:
history = model.fit(train_data,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(test_data, y_test),
                    verbose=1)

Epoch 1/40
313/313 [==============================] - 6s 20ms/step - loss: 0.6880 - acc: 0.5622 - val_loss: 0.6781 - val_acc: 0.6049
Epoch 2/40
313/313 [==============================] - 6s 19ms/step - loss: 0.6619 - acc: 0.6363 - val_loss: 0.6453 - val_acc: 0.6445
Epoch 3/40
313/313 [==============================] - 6s 19ms/step - loss: 0.6302 - acc: 0.6722 - val_loss: 0.6214 - val_acc: 0.6733
Epoch 4/40
313/313 [==============================] - 6s 19ms/step - loss: 0.6103 - acc: 0.6869 - val_loss: 0.6105 - val_acc: 0.6872
Epoch 5/40
313/313 [==============================] - 6s 18ms/step - loss: 0.5983 - acc: 0.6948 - val_loss: 0.6037 - val_acc: 0.6883
Epoch 6/40
313/313 [==============================] - 6s 18ms/step - loss: 0.5897 - acc: 0.7013 - val_loss: 0.6001 - val_acc: 0.6910
Epoch 7/40
313/313 [==============================] - 6s 18ms/step - loss: 0.5832 - acc: 0.7055 - val_loss: 0.5983 - val_acc: 0.6914
Epoch 8/40
313/313 [==============================] - 6s 18ms/step - 